In [27]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [28]:
aniwave = pd.read_csv('raw_data/aniwave_final.csv')
crunchyroll = pd.read_csv('raw_data/crunchyroll.csv')
myanimelist = pd.read_csv('raw_data/mal.csv')

# Data Cleaning

In [29]:
def start_date(aired):
    return aired.partition(' t')[0]

# Get just the start date for myanimelist
myanimelist['Aired'] = myanimelist['Aired'].apply(start_date)

In [30]:
# Rename column names
aniwave = aniwave.rename(columns={'Title':'anime', 'Rating':'rating_aw', 'Vote Count':'votes_aw',
                       'Start Date':'start_date', 'Studio':'studio', 'Start Date':'start_date'})
crunchyroll = crunchyroll.rename(columns={'votes':'votes_cr', 'rate':'rating_cr'})
myanimelist = myanimelist.rename(columns={'Score':'rating_mal', 'English name':'anime', 'Episodes':'episodes',
                                          'Aired':'aired', 'Producers':'producers', 'Licensors':'licensors',
                                          'Studios':'studios', 'Source':'source', 'Duration':'duration',
                                          'Rating':'rating', 'Ranked':'ranked', 'Popularity':'popularity',
                                          'Aired':'start_date', 'Members':'members'})

# Drop shows with missing dates
myanimelist = myanimelist[myanimelist['start_date'] != 'Unknown']
aniwave = aniwave[aniwave['start_date'] != '?']

# Drop shows with missing scores
myanimelist = myanimelist[myanimelist['rating_mal'] != 'Unknown']
myanimelist = myanimelist[myanimelist['episodes'] != 'Unknown']
myanimelist = myanimelist[myanimelist['ranked'] != 'Unknown']
myanimelist = myanimelist[myanimelist['popularity'] != 'Unknown']
myanimelist = myanimelist[myanimelist['members'] != 'Unknown']

# Type conversion
myanimelist['rating_mal'] = myanimelist['rating_mal'].astype(float)
myanimelist['episodes'] = myanimelist['episodes'].astype(float)
myanimelist['ranked'] = myanimelist['ranked'].astype(float)
myanimelist['popularity'] = myanimelist['popularity'].astype(float)

# Date formatting
aniwave['start_date'] = pd.to_datetime(aniwave['start_date'], dayfirst=True, format='mixed')
aniwave['start_date'] = aniwave['start_date'].dt.strftime('%Y-%m-%d')  
myanimelist['start_date'] = pd.to_datetime(myanimelist['start_date'], dayfirst=True, format='mixed')
myanimelist['start_date'] = myanimelist['start_date'].dt.strftime('%Y-%m-%d')

# Standardize scoring (both out of 10)
crunchyroll['rating_cr'] = crunchyroll['rating_cr'] * 2

# Keep only TV shows
aniwave = aniwave.drop(aniwave[aniwave['Type'] != 'TV'].index, axis=0)
myanimelist = myanimelist.drop(myanimelist[myanimelist['Type'] != 'TV'].index, axis=0)

# Drop irrelevant columns
crunchyroll = crunchyroll.drop(['anime_url', 'anime_img', 'episodes', 'weight',
                                'rate_1', 'rate_2', 'rate_3', 'rate_4', 'rate_5'], axis=1)
aniwave = aniwave.drop(['Type'], axis=1)
myanimelist = myanimelist[['anime', 'rating_mal', 'start_date', 'episodes', 'producers', 'licensors',
            'studios', 'source', 'rating', 'ranked', 'members', 'popularity']]

Rename column names
Drop shows with missing dates
Drop shows with missing scores
Type conversion
Date formatting
Standardize scoring (both out of 10)
Keep only TV shows
Drop irrelevant columns

## Record Linkage

### Aniwave - Crunchyroll

In [31]:
import recordlinkage as rl
from recordlinkage.preprocessing import clean

In [32]:
# candidate_links
indexer = rl.Index()
indexer.full()
pairs = indexer.index(aniwave, crunchyroll)

# Make comparisons
compare = rl.Compare()
compare.string('anime', 'anime', method='jarowinkler', threshold=0.95)

# The comparison vectors
compare_vectors = compare.compute(pairs, aniwave, crunchyroll)

In [33]:
# Find matches
compare_vectors.sum(axis=1).value_counts().sort_index(ascending=False)
matches = compare_vectors[compare_vectors.sum(axis=1) == 1]

# Clean matches dataframe 
matches_reset = matches.reset_index()
matches_reset = matches_reset.rename(columns={'level_0':'aniwave_index', 'level_1':'crunchyroll_index'})
matches_reset = matches_reset.drop([0], axis=1)

In [34]:
# Merge Aniwave and Crunchyroll datasets based on the record linkage matches
ani_crunch = pd.merge(matches_reset, aniwave, left_on='aniwave_index', right_index=True, how='inner')
ani_crunch = pd.merge(ani_crunch, crunchyroll, left_on='crunchyroll_index', right_index=True, how='inner')
display(ani_crunch)

ani_crunch.to_excel('merged_data/ani_crunch_recordlinkage.xlsx', index=False)
print(f'Merge success = {len(ani_crunch)/len(crunchyroll) * 100}%')

,aniwave_index,crunchyroll_index,anime_x,rating_aw,votes_aw,start_date,studio,anime_y,votes_cr,rating_cr,...,genre_romance,genre_sci-fi,genre_seinen,genre_sgdrama,genre_shojo,genre_shonen,genre_slice of life,genre_sports,genre_supernatural,genre_thriller
0,37,1135,100% Teacher Pascal,8.15,11.0,2017-04-15,OLM,100% Teacher Pascal,4,6.50,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,41,31,11eyes,5.08,353.0,2009-10-07,Doga Kobo,11eyes,2137,9.16,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,50,602,18if,7.50,175.0,2017-07-07,Gonzo,18if,45,7.06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,53,1149,22/7 (nanabun no nijyuuni),9.12,155.0,2020-01-11,A-1 Pictures,22/7 (nanabun no nijyuuni),3,9.34,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,81,278,91 Days,9.12,1765.0,2016-07-09,Shuka,91 Days,163,9.44,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,11612,866,Z/X IGNITION,7.78,77.0,2014-01-10,Telecom Animation Film,Z/X IGNITION,16,8.00,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
966,11618,722,Zatch Bell!,8.93,496.0,2003-04-06,Toei Animation,Zatch Bell!,29,8.90,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
967,11653,1233,Zo Zo Zombie,7.50,17.0,2020-04-16,J.C.Staff,Zo Zo Zombie,1,2.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
968,11668,976,Zombie-Loan,8.01,89.0,2007-07-04,XEBEC M2,Zombie Loan,10,7.00,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


Merge success = 77.29083665338645%


In [35]:
# find duplicate rows
duplicate_rows = ani_crunch['anime_x'].duplicated()

# print duplicate rows
print(ani_crunch['anime_x'][duplicate_rows==True])

# There are duplicate aniwave shows (should only be crunchyroll because of the one title-mulitple seasons problem)
# REASON: 1) There are duplicate names but of different types (ex: Cells at Work! TV vs ONA) 2) sometimes Crunchyroll has weird stuff (Anohana drama, for instance)
# This should go away when combining with MAL: If I make "type" exact, then it should get rid of these duplicates (which don't really matter: ONA just like a bonus of the main show)

57                   Anohana: The Flower We Saw That Day
199                            DD Fist of the North Star
484                                 LUPIN THE 3rd PART 6
486                               Lupin the Third Part 3
487                               Lupin the Third Part 3
523                           Meow Meow Japanese History
524                           Meow Meow Japanese History
545                                   Mobile Suit Gundam
672             Re:ZERO -Starting Life in Another World-
674    Re:ZERO -Starting Life in Another World- Season 2
676    Re:ZERO -Starting Life in Another World- Season 3
766               Street Fighter II: The Animated Series
796                               The Eccentric Family 2
798                                 The Eccentric Family
916                                          White Album
918                                        White Album 2
942                                         You and Me 2
953                            

### Aniwave - Crunchyroll - Myanimelist

In [36]:
# candidate_links
indexer = rl.Index()
indexer.full()
pairs = indexer.index(ani_crunch, myanimelist)

# Make comparisons
compare = rl.Compare()
compare.exact('start_date', 'start_date', label='start_date')
compare.string('anime_x', 'anime', method='jarowinkler', threshold=0.95, label='anime_aniwave')
compare.string('anime_y', 'anime', method='jarowinkler', threshold=0.95, label='anime_crunchyroll')

# The comparison vectors
compare_vectors = compare.compute(pairs, ani_crunch, myanimelist)
display(compare_vectors)

start_date  anime_aniwave  anime_crunchyroll
0   0               0            0.0                0.0
    2               0            0.0                0.0
    3               0            0.0                0.0
    4               0            0.0                0.0
    5               0            0.0                0.0
...               ...            ...                ...
969 17178           0            0.0                0.0
    17224           0            0.0                0.0
    17229           0            0.0                0.0
    17328           0            0.0                0.0
    17469           0            0.0                0.0

[3685030 rows x 3 columns]

In [37]:
# Find matches
compare_vectors.sum(axis=1).value_counts().sort_index(ascending=False)
matches = compare_vectors[compare_vectors.sum(axis=1) == 3]

# Clean matches dataframe 
matches_reset = matches.reset_index()
matches_reset = matches_reset.rename(columns={'level_0':'ani_crunch_index', 'level_1':'myanimelist_index'})
matches_reset = matches_reset.drop(['anime_aniwave', 'anime_crunchyroll', 'start_date'], axis=1)

In [38]:
# Merge ani_crunch and myanimelist datasets based on the record linkage matches
acm = pd.merge(matches_reset, ani_crunch, left_on='ani_crunch_index', right_index=True, how='inner')
acm = pd.merge(acm, myanimelist, left_on='myanimelist_index', right_index=True, how='inner')
display(acm)

acm.to_excel('merged_data/acm_recordlinkage_pre.xlsx', index=False)
print(f'Merge success = {len(acm)/len(crunchyroll) * 100}%')

,ani_crunch_index,myanimelist_index,aniwave_index,crunchyroll_index,anime_x,rating_aw,votes_aw,start_date_x,studio,anime_y,...,start_date_y,episodes,producers,licensors,studios,source,rating,ranked,members,popularity
0,0,12230,37,1135,100% Teacher Pascal,8.15,11.0,2017-04-15,OLM,100% Teacher Pascal,...,2017-04-15,36.0,"Dentsu, Shogakukan-Shueisha Productions, Asats...",Unknown,OLM,Manga,PG - Children,9080.0,2220,8585.0
1,1,4693,41,31,11eyes,5.08,353.0,2009-10-07,Doga Kobo,11eyes,...,2009-10-07,12.0,"Marvelous, Pony Canyon, feng",Sentai Filmworks,Doga Kobo,Visual novel,R - 17+ (violence & profanity),7510.0,207294,710.0
2,2,12934,50,602,18if,7.50,175.0,2017-07-07,Gonzo,18if,...,2017-07-07,13.0,"Sotsu, DLE, BS Fuji, Tokyo MX, Mobcast, RAY",Funimation,Gonzo,Game,R - 17+ (violence & profanity),7490.0,46598,2353.0
3,4,11650,81,278,91 Days,9.12,1765.0,2016-07-09,Shuka,91 Days,...,2016-07-09,12.0,"Shochiku, Mainichi Broadcasting System, Shochi...","Funimation, Crunchyroll",Shuka,Original,R - 17+ (violence & profanity),752.0,466565,265.0
4,5,5578,90,371,A Certain Magical Index II,9.24,1063.0,2010-10-08,J.C.Staff,A Certain Magical Index,...,2010-10-08,24.0,"Geneon Universal Entertainment, Square Enix, A...",Funimation,J.C.Staff,Light novel,R - 17+ (violence & profanity),1241.0,366426,364.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638,957,12666,11536,519,Yu-Gi-Oh! VRAINS,8.78,693.0,2017-05-10,Gallop,Yu-Gi-Oh! VRAINS,...,2017-05-10,120.0,"TV Tokyo, Nihon Ad Systems",Unknown,Gallop,Card game,PG-13 - Teens 13 or older,5227.0,32679,2845.0
639,960,11648,11575,109,Yuri!!! on ICE,9.23,5321.0,2016-10-06,MAPPA,Yuri!!! on ICE,...,2016-10-06,12.0,"Dentsu, TV Asahi, Movic, Avex Pictures, CIC, C...","Funimation, Crunchyroll",MAPPA,Original,PG-13 - Teens 13 or older,650.0,580801,191.0
640,961,9480,11579,888,Yurikuma Arashi,8.06,195.0,2015-01-06,Silver Link.,Yurikuma Arashi,...,2015-01-06,12.0,"Frontier Works, Media Factory, Mainichi Broadc...",Funimation,SILVER LINK.,Original,R - 17+ (violence & profanity),3321.0,84385,1594.0
641,966,226,11618,722,Zatch Bell!,8.93,496.0,2003-04-06,Toei Animation,Zatch Bell!,...,2003-04-06,150.0,"Yomiko Advertising, Fuji TV","VIZ Media, Flatiron Film Company",Toei Animation,Manga,PG-13 - Teens 13 or older,1434.0,97615,1417.0


Merge success = 51.23505976095617%


## Final Cleaning

In [39]:
# find duplicate rows
duplicate_rows = acm['anime'].duplicated()

# print duplicates
acm['anime'][duplicate_rows==True]

# Find duplicate indices
duplicate_first = (acm['anime'][duplicate_rows==True]).index
duplicates = [*duplicate_first, *(duplicate_first+1)]

# Remove duplicates:
acm = acm.drop(duplicates, axis=0)

In [40]:
# drop duplicate and unnecessary columns
acm = acm.drop(['ani_crunch_index', 'myanimelist_index', 'aniwave_index',
                'crunchyroll_index', 'anime_x','anime_y','start_date_x', 'studio'], axis=1) #there were 2 studio variables

acm = acm.rename(columns={'start_date_y':'start_date'})

performance = ['rating_aw','rating_cr','rating_mal', 'votes_aw','votes_cr', 'members', 'popularity', 'ranked']
biographical = ['start_date','episodes','producers', 'licensors', 'studios','source','rating']
genre = ['genre_action', 'genre_adventure', 'genre_comedy', 'genre_drama',
       'genre_family', 'genre_fantasy', 'genre_food', 'genre_harem',
       'genre_historical', 'genre_horror', 'genre_idols', 'genre_isekai',
       'genre_jdrama', 'genre_magical girls', 'genre_martial arts',
       'genre_mecha', 'genre_music', 'genre_mystery', 'genre_post-apocalyptic',
       'genre_romance', 'genre_sci-fi', 'genre_seinen', 'genre_sgdrama',
       'genre_shojo', 'genre_shonen', 'genre_slice of life', 'genre_sports',
       'genre_supernatural', 'genre_thriller']

# reorder dataframe
acm = acm.loc[:,['anime', *performance, *biographical, *genre]]

print(acm.keys())

Index(['anime', 'rating_aw', 'rating_cr', 'rating_mal', 'votes_aw', 'votes_cr',
       'members', 'popularity', 'ranked', 'start_date', 'episodes',
       'producers', 'licensors', 'studios', 'source', 'rating', 'genre_action',
       'genre_adventure', 'genre_comedy', 'genre_drama', 'genre_family',
       'genre_fantasy', 'genre_food', 'genre_harem', 'genre_historical',
       'genre_horror', 'genre_idols', 'genre_isekai', 'genre_jdrama',
       'genre_magical girls', 'genre_martial arts', 'genre_mecha',
       'genre_music', 'genre_mystery', 'genre_post-apocalyptic',
       'genre_romance', 'genre_sci-fi', 'genre_seinen', 'genre_sgdrama',
       'genre_shojo', 'genre_shonen', 'genre_slice of life', 'genre_sports',
       'genre_supernatural', 'genre_thriller'],
      dtype='object')


In [41]:
display(acm)
print(f'Merge success = {len(acm)/len(crunchyroll) * 100}%')
acm.to_excel('merged_data/acm_recordlinkage_post.xlsx', index=False)

,anime,rating_aw,rating_cr,rating_mal,votes_aw,votes_cr,members,popularity,ranked,start_date,...,genre_romance,genre_sci-fi,genre_seinen,genre_sgdrama,genre_shojo,genre_shonen,genre_slice of life,genre_sports,genre_supernatural,genre_thriller
0,100% Teacher Pascal,8.15,6.50,5.72,11.0,4,2220,8585.0,9080.0,2017-04-15,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,11eyes,5.08,9.16,6.16,353.0,2137,207294,710.0,7510.0,2009-10-07,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,18if,7.50,7.06,6.17,175.0,45,46598,2353.0,7490.0,2017-07-07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,91 Days,9.12,9.44,7.84,1765.0,163,466565,265.0,752.0,2016-07-09,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,A Certain Magical Index II,9.24,7.10,7.61,1063.0,103,366426,364.0,1241.0,2010-10-08,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,Yu-Gi-Oh! GX,8.55,8.48,7.16,711.0,45,162129,917.0,2966.0,2004-10-06,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
639,Yuri!!! On ICE,9.23,9.66,7.92,5321.0,673,580801,191.0,650.0,2016-10-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
640,Yurikuma Arashi,8.06,9.42,7.08,195.0,14,84385,1594.0,3321.0,2015-01-06,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
641,Zatch Bell!,8.93,8.90,7.55,496.0,29,97615,1417.0,1434.0,2003-04-06,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Merge success = 49.64143426294821%
